In [6]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import csv

In [7]:
# Get all the stock symbols
mypath = '.\\stock_prices\\Data\Stocks\\'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
stocks = []
for file in onlyfiles:
    stocks.append(file[:-7])

# Match stock symbols with company names
symbol_to_name = {}
name_to_symbol = {}
stop_list = [' inc', ' inc', ', inc', ',', '.', ' ltd', ' lp', '\n']
with open('./stock_name_symbol.txt') as nameFile:
    for line in nameFile:
        tempLine = line.split(',')
        symbol = tempLine[0].lower()
        name = tempLine[1].lower()
        for string in stop_list:
            name = name.replace(string, '')
        if tempLine[0].lower() in stocks:
            symbol_to_name[symbol] = name
            name_to_symbol[name] = symbol

In [10]:
# Get a list of all named stocks that have metrics in our dataset
name_to_secID = {}
secID_to_name = {}

df = pd.read_csv('.\\company_metrics\\companies-names-industries.csv')
for name, sec_id in zip(df['Latest Name'], df['SEC ID']):
    tempName = name.lower()
    for string in stop_list:
        tempName = tempName.replace(string, '')
    if tempName in name_to_symbol:
        name_to_secID[tempName] = sec_id
        secID_to_name[sec_id] = tempName

In [14]:
# Get the open, close, high, low, and volume of stock trades for each company that has metrics
stock_prices = {}
for stockName in name_to_secID:
    stock_prices[stockName] = []
    stockSymbol = name_to_symbol[stockName]
    fileName = '.\\stock_prices\\Stocks\\'+stockSymbol+'.us.txt'
    with open(fileName) as file:
        for line in file:
            tempLine = line.split(',')
            if tempLine[0] != 'Date':
                stock_prices[stockName].append(tempLine)

In [15]:
### Get the company metrics for each company in our dataset ###
metrics = {}

# Assets
assets = {}
with open('.\\company_metrics\\metrics\\Assets-quarterly.csv') as file:
    for line in file:
        tempLine = line.split(',')
        if tempLine[0] != 'SEC ID':
            if int(tempLine[0]) in list(secID_to_name.keys()):
                assets[int(tempLine[0])] = tempLine[1:-1]
metrics['assets'] = assets

# Cash and cash equivalents at carrying value
cash = {}
with open('.\\company_metrics\\metrics\\CashAndCashEquivalentsAtCarryingValue-quarterly.csv') as file:
    for line in file:
        tempLine = line.split(',')
        if tempLine[0] != 'SEC ID':
            if int(tempLine[0]) in list(secID_to_name.keys()):
                cash[int(tempLine[0])] = tempLine[1:-1]
metrics['cash'] = cash
                
# Liabilities
liabilities = {}
with open('.\\company_metrics\\metrics\\Liabilities-quarterly.csv') as file:
    for line in file:
        tempLine = line.split(',')
        if tempLine[0] != 'SEC ID':
            if int(tempLine[0]) in list(secID_to_name.keys()):
                liabilities[int(tempLine[0])] = tempLine[1:-1]
metrics['liabilities'] = liabilities

# Operating Expenses
operating_expenses = {}
with open('.\\company_metrics\\metrics\\OperatingIncomeLoss-quarterly.csv') as file:
    for line in file:
        tempLine = line.split(',')
        if tempLine[0] != 'SEC ID':
            if int(tempLine[0]) in list(secID_to_name.keys()):
                operating_expenses[int(tempLine[0])] = tempLine[1:-1]
metrics['operating_expenses'] = operating_expenses

# Property Plant and Equipment Net
property_value = {}
with open('.\\company_metrics\\metrics\\PropertyPlantAndEquipmentNet-quarterly.csv') as file:
    for line in file:
        tempLine = line.split(',')
        if tempLine[0] != 'SEC ID':
            if int(tempLine[0]) in list(secID_to_name.keys()):
                property_value[int(tempLine[0])] = tempLine[1:-1]
metrics['property_value'] = property_value

# Revenues
revenue = {}
with open('.\\company_metrics\\metrics\\Revenues-quarterly.csv') as file:
    for line in file:
        tempLine = line.split(',')
        if tempLine[0] != 'SEC ID':
            if int(tempLine[0]) in list(secID_to_name.keys()):
                revenue[int(tempLine[0])] = tempLine[1:-1]
metrics['revenue'] = revenue

In [16]:
# Get companies that have a complete dataset
all_data_comps = list(secID_to_name.keys())
for metric in metrics:
    for secID in metrics[metric]:
        if '' in metrics[metric][secID]:
            if secID in all_data_comps:
                del all_data_comps[all_data_comps.index(secID)]

for secID in all_data_comps:
    if secID in metrics['assets']:
        print("Name: " + secID_to_name[secID])
        print("SEC ID: " + str(secID))
        print("# of stock datapoints: "+ str(len(stock_prices[secID_to_name[secID]])))
        print(len(metrics['assets'][secID]))

Name: ebay
SEC ID: 1065088
# of stock datapoints: 4814
25
Name: sirius xm holdings
SEC ID: 908937
# of stock datapoints: 3201
25
Name: aegion corp
SEC ID: 353020
# of stock datapoints: 1521
25
Name: nektar therapeutics
SEC ID: 906709
# of stock datapoints: 3198
25
Name: cme group
SEC ID: 1156375
# of stock datapoints: 3201
25
Name: biomarin pharmaceutical
SEC ID: 1048477
# of stock datapoints: 3199
25
Name: scientific games corp
SEC ID: 750004
# of stock datapoints: 3201
25
Name: shire plc
SEC ID: 936402
# of stock datapoints: 3201
25


In [32]:
# Get Ebay Data for a quick and dirty analysis
tempID = 936402
print("SEC ID,2011Q1,2011Q2,2011Q3,2011Q4,2012Q1,2012Q2,2012Q3,2012Q4,2013Q1,2013Q2,2013Q3,2013Q4,2014Q1,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3,2016Q4,2017Q1,2017Q2")
for metric in metrics:
    print(metric)
    printStr = str(tempID)+','
    for item in metrics[metric][tempID]:
        printStr += str(item) + ','
    print(printStr[:-1])

'''
for day in stock_prices['sirius xm holdings']:
    printStr = ''
    for item in day:
        printStr += str(item.replace('\n', '')) + ','
    print(printStr[:-1])
'''

SEC ID,2011Q1,2011Q2,2011Q3,2011Q4,2012Q1,2012Q2,2012Q3,2012Q4,2013Q1,2013Q2,2013Q3,2013Q4,2014Q1,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3,2016Q4,2017Q1,2017Q2
revenue
936402,972200000.000,1062900000.000,1086100000.000,1142200000.000,1171800000.000,1207800000.000,1100400000.000,1201200000.000,1161900000.000,1274500000.000,1236600000.000,1261300000.000,1346800000.000,1502100000.000,1597100000.000,1576100000.000,1488400000.000,1557600000.000,1655000000.000,1715700000.000,1709300000.000,2429100000.000,3452100000.000,3806100000.000,3572300000.000
cash
936402,712000000.000,144600000.000,276400000.000,620000000.000,879400000.000,1112700000.000,1321900000.000,1482200000.000,1450700000.000,1301900000.000,1686100000.000,2239400000.000,139100000.000,153600000.000,467700000.000,2982400000.000,74300000.000,64000000.000,44900000.000,135500000.000,69000000.000,693400000.000,728600000.000,528800000.000,369000000.000
property_value
936402,869800000.000,905800000.000,918800

"\nfor day in stock_prices['sirius xm holdings']:\n    printStr = ''\n    for item in day:\n        printStr += str(item.replace('\n', '')) + ','\n    print(printStr[:-1])\n"